In [1]:
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
import gensim
import time, math
import numpy as np
import torch
import torch.nn as nn
import io
from torch.autograd import Variable

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shijiez/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/Users/shijiez/environments/pytorch/lib/python3.6/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


In [2]:
newsData = pd.read_csv("stockNews/Combined_News_DJIA.csv")

In [3]:
pd.set_option('display.max_colwidth', 2000)

In [4]:
newsTrain = newsData[newsData['Date']< '2015-01-01']
newsTest = newsData[newsData['Date']>'2014-12-31']
labelTrain = newsTrain['Label'].values
labelTest = newsTest['Label'].values
newsTrain.head(2)

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as countries move to brink of war""",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into South Ossetia; footage from fighting (YouTube)',"b'Russian tanks are moving towards the capital of South Ossetia, which has reportedly been completely destroyed by Georgian artillery fire'","b""Afghan children raped with 'impunity,' U.N. official says - this is sick, a three year old was raped and they do nothing""",b'150 Russian tanks have entered South Ossetia whilst Georgia shoots down two Russian jets.',"b""Breaking: Georgia invades South Ossetia, Russia warned it would intervene on SO's side""","b""The 'enemy combatent' trials are nothing but a sham: Salim Haman has been sentenced to 5 1/2 years, but will be kept longer anyway just because they feel like it.""",...,"b'Georgia Invades South Ossetia - if Russia gets involved, will NATO absorb Georgia and unleash a full scale war?'",b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to prevent an Israeli strike on Iran."" Israeli Defense Minister Ehud Barak: ""Israel is prepared for uncompromising victory in the case of military hostilities.""'",b'This is a busy day: The European Union has approved new sanctions against Iran in protest at its nuclear programme.',"b""Georgia will withdraw 1,000 soldiers from Iraq to help fight off Russian forces in Georgia's breakaway region of South Ossetia""",b'Why the Pentagon Thinks Attacking Iran is a Bad Idea - US News &amp; World Report',b'Caucasus in crisis: Georgia invades South Ossetia',"b'Indian shoe manufactory - And again in a series of ""you do not like your work?""'",b'Visitors Suffering from Mental Illnesses Banned from Olympics',"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,"b'Why wont America and Nato help us? If they wont help us now, why did we help them in Iraq?'",b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli training, we're fending off Russia """,b'Georgian army flees in disarray as Russians advance - Gori abandoned to Russia without a shot fired',"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zealand Passports doing in Iraq?',b'Russia angered by Israeli military sale to Georgia',b'An American citizen living in S.Ossetia blames U.S. and Georgian leaders for the genocide of innocent people',...,b'Israel and the US behind the Georgian aggression?',"b'""Do not believe TV, neither Russian nor Georgian. There are much more victims""'",b'Riots are still going on in Montreal (Canada) because police murdered a boy on Saturday.',b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Torture',b' Russia has just beaten the United States over the head with Peak Oil',b'Perhaps *the* question about the Georgia - Russia conflict ',b'Russia is so much better at war',"b""So this is what it's come to: trading sex for food."""


In [5]:
def news_to_words( news ):
    # 1. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", news) 
    #
    # 2. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 3. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 4. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 5. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))

In [6]:
trainHeadLines = []
for row in range(0,len(newsTrain.index)):
#for row in range(0,2):
    #pre-process words
    combinedNews = " ".join(str(x).replace("b\"","").replace("b\'","") for x in newsTrain.iloc[row,2:27])
    processedNews = news_to_words(combinedNews)
    trainHeadLines.append(processedNews)
# print(trainheadlines)
#print(len(newsTrain.index))

testHeadLines = []
for row in range(0,len(newsTest.index)):
#for row in range(0,2):
    #pre-process words
    combinedNews = " ".join(str(x).replace("b\"","").replace("b\'","") for x in newsTest.iloc[row,2:27])
    processedNews = news_to_words(combinedNews)
    testHeadLines.append(processedNews)
# print(trainheadlines)
#print(len(newsTrain.index))

# Processing, split sentences to list

In [7]:
tmpHeadLines = []
for headLine in trainHeadLines:
    headLineSplit = headLine.split(' ')
    tmpHeadLines.append(headLineSplit)
trainHeadLines = tmpHeadLines

tmpHeadLines = []
for headLine in testHeadLines:
    headLineSplit = headLine.split(' ')
    tmpHeadLines.append(headLineSplit)
testHeadLines = tmpHeadLines
del tmpHeadLines

In [8]:
# Load Google's pre-trained Word2Vec model.
model = gensim.models.KeyedVectors.load_word2vec_format('/Users/shijiez/word2vec/GoogleNews-vectors-negative300.bin', binary=True)  
wv = model.wv
del model

/Users/shijiez/environments/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


# Check vocabulary size
over 5000 words not found, initialized to normal distributed

In [9]:
vocabulary = []
for line in trainHeadLines:
    for word in line:
        if word not in vocabulary:
            vocabulary.append(word)

for line in testHeadLines:
    for word in line:
        if word not in vocabulary:
            vocabulary.append(word)


In [10]:
emb_dim = 300
matrix_len = len(vocabulary)
weights_matrix = np.zeros((matrix_len, emb_dim))
words_found = 0

for i, word in enumerate(vocabulary):
    try: 
        weights_matrix[i] = wv[word]
        words_found += 1
    except KeyError:
        weights_matrix[i] = np.random.normal(scale=0.6, size=(emb_dim, ))

weights_matrix = torch.FloatTensor(weights_matrix)

# Method to create embedding layer
This method by default, assume the embedding layer is trainable

In [11]:
def create_emb_layer(weights_matrix, non_trainable=False):
    num_embeddings, embedding_dim = weights_matrix.size()
    emb_layer = nn.Embedding(num_embeddings, embedding_dim)
    emb_layer.load_state_dict({'weight': weights_matrix})
    if non_trainable:
        emb_layer.weight.requires_grad = False

    return emb_layer, num_embeddings, embedding_dim

create_emb_layer(weights_matrix)

(Embedding(33136, 300), 33136, 300)

In [12]:
# Turn words list into list of longs
def word_tensor(sentenceList):
    tensor = torch.zeros(len(sentenceList)).long()
    for c in range(len(sentenceList)):
        tensor[c] = vocabulary.index(sentenceList[c])
    return Variable(tensor)

print(word_tensor(trainHeadLines[0]))

tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,   3,  22,  23,  24,  25,  17,
         18,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  34,  44,   3,  22,  45,  17,  18,  46,   0,
         47,   2,   3,  48,   9,   0,  49,  17,  18,  12,  50,  51,  52,  53,
         54,  55,  56,  44,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,
         29,  15,  67,  68,  25,  69,  70,  71,  72,  73,  74,  75,  36,  76,
          0,   8,  12,  77,  78,  79,  80,  81,  82,  83,  39,  36,  84,  85,
         86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  12,   0,   8,  96,
         97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110,
        111,   5, 112, 113,   8, 114, 115,  13,   0,  49,  17,  18,  12, 116,
        117, 118, 119,   0, 120, 121, 122,   8, 123, 124, 125, 126, 127, 128,
         77, 129,  51, 130, 131,  85, 132,  83,  85, 133, 134, 1

Function to expand label for a sequence, to be equally long, and convert to pytorch format.

In [13]:
def label_tensor(label, length):
    tensor = torch.ones(length).long()
    tensor = tensor * label
    return Variable(tensor)

# Convert dataset to pytorch format
1. convert array of words in the string to array of indecis representing the entry in vector space.
2. expand single label to be the same as length array, and convert to pytorch format  
Return 2 items, X and y.

In [14]:
def convert_to_pytorch_format(X, y):
    M = len(X)
    X_processed = []
    y_processed = []

    for i in range(M):
        sequenceLength = len(X[i])
        inp = word_tensor(X[i])
        target = label_tensor(y[i], sequenceLength)
        X_processed.append(inp)
        y_processed.append(target)
    return X_processed, y_processed

In [15]:
(X_train, y_train) = convert_to_pytorch_format(trainHeadLines, labelTrain)
(X_test, y_test) = convert_to_pytorch_format(testHeadLines, labelTest)

# Training

In [16]:
class RNN(nn.Module):
    # define model architecture
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        
        (self.embedding, num_embeddings, embedding_dim) = create_emb_layer(weights_matrix, non_trainable=False)

        self.input_size = embedding_dim
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        # encoder, GRU, decode
        # self.embeddings = nn.Embedding.from_pretrained(weights)
        # self.encoder = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(input_size, hidden_size, n_layers)
        self.model = nn.Linear(hidden_size, output_size)
    
    # forward prop definition  FOR A SINGLE ELEMENT IN THE SEQUENCE
    def forward(self, input, hidden):
        input = self.embedding(input.view(1, -1))

        output, hidden = self.gru(input.view(1, 1, -1), hidden)
        output = self.model(output.view(1, -1))
        return output, hidden

    # Hidden state initialization for a single sequence
    def init_hidden(self):
        return Variable(torch.zeros(self.n_layers, 1, self.hidden_size))
    
    # forward propagate and return last layer softmax output
    def output_softmax(self, line_tensor):
        hidden = self.init_hidden()
        # loop thru sequences, make continuous prediction
        for i in range(line_tensor.size()[0]):
            output, hidden = self(line_tensor[i], hidden)
        return output
    
    
    # predict function
    def predict_dataset(self, dataset):
        predictions = []
        for i in range(len(dataset)):
            output = self.output_softmax(dataset[i])
            predictedLabel = output.argmax()
            predictions.append(predictedLabel)
        return np.array(predictions)
    
    def accuracy(self, dataset, labels):
        m = len(labels)
        predictions = self.predict_dataset(dataset)
        correctCount = np.sum(predictions == labels)
        accuracy = correctCount/m
        return accuracy

In [17]:
def train(inp, target):
    string_len = len(inp)
    hidden = model.init_hidden()
    model.zero_grad()
    loss = 0

    for c in range(string_len):
        output, hidden = model(inp[c], hidden)
        loss += criterion(output, target[c].unsqueeze(0))

    loss.backward()
    model_optimizer.step()
    # print(loss.data)
    # print(loss.data.item())
    
    return loss.data.item() / string_len

In [18]:
def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [22]:
testSampleNo = 0
trainSampleNo = 0

n_epochs = 1
print_every = 100
evaluate_every = 10
hidden_size = 100
vector_size = 300
output_size = 2
n_layers = 1
lr = 0.005
trainM = len(X_train)
testM = len(X_test)

model = RNN(vector_size, hidden_size, output_size, n_layers)
model_optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

start = time.time()
all_losses = []
loss_avg = 0

for epoch in range(1, n_epochs + 1):
    for i in range(trainM):
        loss = train(X_train[i], y_train[i])
        loss_avg += loss
        if i % evaluate_every == 0:
            print(i , " step, accuracy = ", model.accuracy(X_test, labelTest))

#         if epoch % print_every == 0:
#             print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
#             print(evaluate('Wh', 100), '\n')

#         if epoch % plot_every == 0:
#             all_losses.append(loss_avg / plot_every)
#             loss_avg = 0

0  step, accuracy =  0.4947089947089947
10  step, accuracy =  0.455026455026455
20  step, accuracy =  0.5052910052910053
30  step, accuracy =  0.5105820105820106
40  step, accuracy =  0.4973544973544973
50  step, accuracy =  0.49206349206349204
60  step, accuracy =  0.4947089947089947
70  step, accuracy =  0.4973544973544973
80  step, accuracy =  0.5026455026455027
90  step, accuracy =  0.5
100  step, accuracy =  0.48677248677248675
110  step, accuracy =  0.48412698412698413
120  step, accuracy =  0.4894179894179894
130  step, accuracy =  0.4947089947089947
140  step, accuracy =  0.49206349206349204
150  step, accuracy =  0.4894179894179894
160  step, accuracy =  0.43386243386243384
170  step, accuracy =  0.5026455026455027
180  step, accuracy =  0.5079365079365079
190  step, accuracy =  0.49206349206349204
200  step, accuracy =  0.49206349206349204
210  step, accuracy =  0.5185185185185185
220  step, accuracy =  0.5079365079365079
230  step, accuracy =  0.5026455026455027
240  step, a

## 2nd epoch

In [ ]:
for epoch in range(1, n_epochs + 1):
    for i in range(trainM):
        loss = train(X_train[i], y_train[i])
        loss_avg += loss
        if i % evaluate_every == 0:
            print(i , " step, accuracy = ", model.accuracy(X_test, labelTest))

0  step, accuracy =  0.5370370370370371
10  step, accuracy =  0.5132275132275133
20  step, accuracy =  0.4947089947089947
30  step, accuracy =  0.48148148148148145
40  step, accuracy =  0.48148148148148145
50  step, accuracy =  0.5052910052910053
60  step, accuracy =  0.5423280423280423
70  step, accuracy =  0.48677248677248675
80  step, accuracy =  0.5264550264550265
90  step, accuracy =  0.5661375661375662
100  step, accuracy =  0.5052910052910053
110  step, accuracy =  0.5026455026455027
120  step, accuracy =  0.5264550264550265
130  step, accuracy =  0.5158730158730159
140  step, accuracy =  0.49206349206349204
150  step, accuracy =  0.5158730158730159
160  step, accuracy =  0.4708994708994709
170  step, accuracy =  0.5052910052910053
180  step, accuracy =  0.4947089947089947
190  step, accuracy =  0.5211640211640212
200  step, accuracy =  0.47619047619047616
210  step, accuracy =  0.5079365079365079
220  step, accuracy =  0.49206349206349204
230  step, accuracy =  0.49206349206349